**安装并导入相关包**

In [ ]:
!pip install datasets
!pip install trl

**自监督模式微调方法：输入一段文字，对每个进行预测。**

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM
from trl import SFTConfig, SFTTrainer

In [ ]:
dataset = load_dataset('/imdb')

In [ ]:
small_train_dataset = dataset["train"].select(range(10))
small_eval_dataset = dataset["test"].select(range(10))

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

training_args = SFTConfig(
    eval_strategy="epoch",
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
    logging_steps=1,
)
trainer = SFTTrainer(
    model,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    args=training_args,
)
trainer.train()

**问答模式微调方法：question: text1, answer: text2, 模型对text2的每个字进行预测。**



In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k")

In [8]:
dataset1 = dataset['train'].train_test_split(test_size=0.3)
small_train_dataset = dataset1['train'].select(range(20))
small_val_dataset = dataset1['test'].select(range(20))

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

training_args = SFTConfig(
    eval_strategy="epoch",
    output_dir="/tmp",
    logging_steps=1,
)

trainer = SFTTrainer(
    model,
    train_dataset=small_train_dataset,
    eval_dataset=small_val_dataset,
    args=training_args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)
trainer.train()

**对话模式语言模型：输入human: text1, assistant: text2, human: text3, assistant: text4, human: text5, assistant: text6, ..., 模型对assistant对应的text的每个字进行预测。**

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, DatasetDict
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
import re
import random
from multiprocessing import cpu_count

In [ ]:
from huggingface_hub import login
login(token="here is your auth token")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B-Instruct')
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-1B-Instruct')

In [ ]:
# 加载数据
dataset = load_dataset("HuggingFaceH4/ultrachat_200k")
dataset = DatasetDict({
    "train": dataset["train_sft"],
    "test": dataset["test_sft"]
})

In [14]:
dataset['train'][0]

{'prompt': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
 'prompt_id': 'f0e37e9f7800261167ce91143f98f511f768847236f133f2d0aed60b444ebe57',
 'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of 

In [ ]:
# 处理数据
chat_template =
"{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n
{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n
{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n
{% endif %}\n{% if loop.last and add_generation_prompt %}\n
{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = chat_template
def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)
    return example
dataset1 = dataset.map(
            apply_chat_template,
            num_proc=cpu_count(),
            fn_kwargs={"tokenizer": tokenizer},
            remove_columns=list(dataset["train"].features),
            desc="Applying chat template",
      )
small_train_dataset = dataset1["train"].select(range(10))
small_test_dataset = dataset1["test"].select(range(10))

In [ ]:
dataset1['train'][0]

{'text': "<|system|>\n<|eot_id|>\n<|user|>\nThese instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?<|eot_id|>\n<|assistant|>\nThis feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.<|eot_id|>\n<|user|>\nCan you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?<|eot_id|>\n<|assistant|>\nSure, here are the st

In [ ]:
dataset['train'][0:2]

{'prompt': ["These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
  'Which famous landmarks should I visit in London, beyond the usual ones?'],
 'prompt_id': ['f0e37e9f7800261167ce91143f98f511f768847236f133f2d0aed60b444ebe57',
  'f5025bdcae61bb77fd98a4d6cd6ba8e0199a098cfebcf6830f4a85e0d13a9e21'],
 'messages': [[{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version a

In [ ]:
dataset1['train']

Dataset({
    features: ['text'],
    num_rows: 207865
})

In [ ]:
dataset1['train'][0:1]

{'text': ["<|system|>\n<|eot_id|>\n<|user|>\nThese instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?<|eot_id|>\n<|assistant|>\nThis feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.<|eot_id|>\n<|user|>\nCan you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?<|eot_id|>\n<|assistant|>\nSure, here are the s

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 训练数据
training_args = SFTConfig(
    eval_strategy="epoch",
    output_dir="/tmp",
    overwrite_output_dir=True,
    dataset_text_field="text",
    packing=True,
    logging_steps=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
)


trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_test_dataset,
        tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fengguoying85 (fengguoying85-pangan). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.964400,1.795119
2,0.576000,1.940404
3,0.565600,2.052275


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


TrainOutput(global_step=42, training_loss=0.9038063607045582, metrics={'train_runtime': 2697.136, 'train_samples_per_second': 0.016, 'train_steps_per_second': 0.016, 'total_flos': 251118407909376.0, 'train_loss': 0.9038063607045582, 'epoch': 3.0})

SFT之前推理

In [ ]:
from huggingface_hub import login
login(token="here is your token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a frendly teacher, who always reply in a gentle style!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "I'm so glad you asked. My name is here to help, and I'm a friendly AI teacher. I'm designed to provide information, answer questions, and assist with tasks in a gentle and supportive way. I'm always here to help you learn and grow, and I'm excited to get to know you better.\n\nI don't have a personal name, but you can think of me as your virtual guide or study buddy. I'll do my best to make learning fun and engaging, and I'll always be here to offer a helping hand or a reassuring word.\n\nSo, what would you like to talk about or learn about today?"}


In [ ]:
# sft之后推理
model_id = "/tmp/checkpoint-6/"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a frendly teacher, who always reply in a gentle style!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

youtue看到的数据集 备份这里

In [ ]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


openassistant_best_replies_train.jsonl:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

openassistant_best_replies_eval.jsonl:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [ ]:
dataset[0:3]

{'text': ['### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining p